# LLM Fine-tuning BERT for Text Classification Pipeline Notebook

UPDATE COLAB BADGE LINK
<a target="_blank" href="https://colab.research.google.com/github/unionai-oss/fine-tune-bert-sentiment-classifier/blob/main/bert-fine-tune.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

GitHub repository: https://github.com/unionai-oss/object-detection-computer-vision-pipeline

This notebook is a pipeline for fine-tuning a fast rcnn model on a custom dataset with PyTorch

## Project Setup:

Sign up for Union while libraries are installing below:

- Union sign up: https://signup.union.ai/
- Union Dashboard: https://serverless.union.ai/

Install python libraries by running the code cell below:

!pip install

Auth Union account

In [ ]:
!union create login --auth device-flow

## Build a Simple Workflow

In [ ]:
%%writefile simple_wf.py

# Import libraries and modules
# task
from flytekit import task, workflow

@task
def hello_world(name: str) -> str:
    return f"Hello {name}"

# workflow
@workflow
def main(name: str) -> str:
    return hello_world(name=name)

In [ ]:
# Run locally
!union run simple_wf.py main --name Flyte

In [ ]:
# Run on Union
!union run --remote simple_wf.py main --name Flyte

In [ ]:
!union run simple_wf.py main --help

In [ ]:
!union run --help

## Fine tune Fast RCNN Model for Object Detection
- Download the dataset
- Download the model
- Train the model
- Evaluate the model

Use the model on new images or video

In [ ]:
%%writefile workflow.py